In [1]:
import os
import numpy as np
import tifffile
from tqdm import tqdm

# Generate training data from image planes

In [2]:
# Define paths

# Models
path = r'/home/hmorales/WorkSpace/DataIsoReconstructions/nuclei2nuclei_leftdistorted/'

# Parameters for the patch selection
patch_size = 128
stride = 128
signal_intensity_threshold = 20000  #parameter for selecting image patches containing signals

xy_interval=2
xz_interval=4


# Define input and ouput directorires 

raw_data_path   = os.path.join(path, 'raw_data/')
train_data_path = os.path.join(path, 'train_data/')

if not os.path.exists(train_data_path):
    os.mkdir(train_data_path)

xy_path    = os.path.join(raw_data_path, 'xy/')
xy_lr_path = os.path.join(raw_data_path,'xy_lr/')
xz_path    = os.path.join(raw_data_path,'xz/')

In [3]:
# Initialize arrays

xy = []
xy_lr = []
xz = []

# Loop over lateral images
file_list_xy = os.listdir(xy_path)
for i in tqdm(range(0, len(file_list_xy), xy_interval)):
    xy_img = tifffile.imread(xy_path + str(i + 1) + '.tif')
    xy_lr_img = tifffile.imread(xy_lr_path + str(i + 1) + '.tif')
    L0 = min(xy_img.shape[0], xy_lr_img.shape[0])
    L1 = min(xy_img.shape[1], xy_lr_img.shape[1])
    for m in range(0, L0 - patch_size + 1, stride):
        for n in range(0, L1 - patch_size + 1, stride):
            crop_xy    =    xy_img[m:m + patch_size, n:n + patch_size]
            crop_xy_lr = xy_lr_img[m:m + patch_size, n:n + patch_size]
            
            if np.max(crop_xy) >= signal_intensity_threshold:
                xy.append(crop_xy)
                xy_lr.append(crop_xy_lr)

# Loop over axial images   
file_list_xz = os.listdir(xz_path)
for i in tqdm(range(0, len(file_list_xz), xz_interval)):
    xz_img = tifffile.imread(xz_path + str(i + 1) + '.tif')
    for m in range(0, xz_img.shape[0] - patch_size + 1, stride):
        for n in range(0, xz_img.shape[1] - patch_size + 1, stride):
            crop_xz = xz_img[m:m + patch_size, n:n + patch_size]

            if np.max(crop_xz) >= signal_intensity_threshold:
                xz.append(crop_xz)


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4764/4764 [00:07<00:00, 600.17it/s]


In [4]:
# Convert to arays and save

xy = np.array(xy, dtype=np.float32)
xy_lr = np.array(xy_lr, dtype=np.float32)
xz = np.array(xz, dtype=np.float32)
print(xy.shape, xy_lr.shape, xz.shape)

np.savez(os.path.join(train_data_path, 'train_data.npz'), xy=xy, xy_lr=xy_lr, xz=xz)

(6447, 128, 128) (6447, 128, 128) (22309, 128, 128)


In [5]:
#Save tiff to double check

import tifffile as tiff
tiff.imwrite(os.path.join(train_data_path,'xy.tif'), xy)
tiff.imwrite(os.path.join(train_data_path,'xy_lr.tif'), xy_lr)
tiff.imwrite(os.path.join(train_data_path,'xz.tif'), xz)